In [ ]:

## finish this off on the side!!! 
## next steps build function off the best result

## and test it on the test batch

## give a graph that shows y_actual vs y_predict

## save a model for every parameter you care about --- like flight id or route or 



In [15]:
hi[0]

,mean_r2,mean_MAPE,mean_MSE,std_r2,std_MAPE,std_MSE,score,alpha,intercept,tol,...,combined_score,opti_run,n_estimators,max_depth,min_samples_split,min_samples_leaf,random_state,learning_rate,depth,l2_leaf_reg
21,0.078277,4.941517e+16,70.050639,0.019678,6.309893e+14,13.722034,5,99.934116,True,0.112348,...,64.567257,ridge_all_features,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0.078210,4.991278e+16,70.055631,0.020734,7.072129e+14,13.732237,5,99.992802,True,0.059643,...,64.576558,ridge_lasso_features,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.104810,2.864678e+16,68.859243,0.016020,2.842307e+15,12.656255,5,NaN,NaN,NaN,...,61.642082,ran_forest_all_features,125.0,19.0,6.0,9.0,NaN,NaN,NaN,NaN
1,0.097609,2.903251e+16,69.153410,0.012340,3.035956e+15,12.781126,5,NaN,NaN,NaN,...,62.403390,ran_forest_lasso_features,131.0,11.0,7.0,8.0,NaN,NaN,NaN,NaN
26,0.061302,3.210326e+16,70.430978,0.019263,2.817322e+15,12.424414,5,NaN,NaN,NaN,...,66.113415,Dec_Tree_all_features,NaN,4.0,7.0,10.0,50.0,NaN,NaN,NaN
21,0.052449,3.238770e+16,70.802836,0.014550,2.616372e+15,12.685494,5,NaN,NaN,NaN,...,67.089328,Dec_Tree_lasso_features,NaN,4.0,4.0,10.0,10.0,NaN,NaN,NaN
25,0.052449,3.238770e+16,70.802836,0.014550,2.616372e+15,12.685494,5,NaN,NaN,NaN,...,67.089328,Dec_Tree_lasso_features,NaN,4.0,4.0,10.0,22.0,NaN,NaN,NaN
23,0.720789,2.364996e+16,35.464243,0.104638,8.979420e+14,3.465619,3,NaN,NaN,NaN,...,9.901991,catBoost_all_features,NaN,NaN,NaN,NaN,NaN,0.084313,9.0,0.249099
27,0.775143,2.340971e+16,31.835973,0.085024,8.270877e+14,3.282729,3,NaN,NaN,NaN,...,7.158535,catBoost_lasso_features,NaN,NaN,NaN,NaN,NaN,0.099768,10.0,0.906476


In [14]:
hi = run_regression_models(X_train_scaled, y_train, featurenames)
    # pull the concat best data
    # pull out the features that were lassoed 
    #OUTPUT ===== return concat_best_data, lassoed_features
    
    ## pull out features that are used.. 

c:\Users\erick\gitrepositories\flight_delays_ML\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.118e+04, tolerance: 4.157e+03
  model = cd_fast.enet_coordinate_descent(
[I 2023-09-06 10:20:01,811] A new study created in memory with name: RidgeRegression Optimization
[I 2023-09-06 10:20:01,895] Trial 0 finished with value: 69.16957849589886 and parameters: {'alpha': 20.160126965802917, 'fit_intercept': False, 'tol': 0.002515536674290861, 'solver': 'svd'}. Best is trial 0 with value: 69.16957849589886.
[I 2023-09-06 10:20:02,469] Trial 1 finished with value: 64.6297222551626 and parameters: {'alpha': 12.482762901443367, 'fit_intercept': True, 'tol': 0.002268324154354905, 'solver': 'sag'}. Best is trial 1 with value: 64.6297222551626.
[I 2023-09-06 10:20:03,247] Trial 2 finished w

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import math
import catboost
from catboost import CatBoostRegressor

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna 

# Feel free to add all the libraries you need
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from geopy.distance import geodesic

In [2]:
df1 = pd.read_csv('data/ordinalencoded.csv', index_col=0)
df = df1[df1['international_flight']==0]
# define features and target
# if you want to create a polynomial you do so at the start and recreate the dataframe
features = df.columns.tolist()
features.remove('target')
X = df[features]
y = df['target']
featurenames = list(X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12, test_size=0.5)
#df1 = pd.read_csv('data/ordinalencoded_expandeddataset.csv', index_col=0)

In [3]:
## scaler is applied outside of function
# standard scaler 
scaler = StandardScaler()
#scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:

## pull the column names immediately they are useful
### new definition
def run_regression_models(x234_array, y234_array, features):
## applies lasso optimization to the data set for feature selection
## you have two trains --- lassoed_train & X_train 
## make sure you make a list of the feature in the lassoed train... 

## how many of each iteration do you want 
    ridge_iter = 30
    ran_forest_iter = 10
    SVR_iter = 1
    dec_tree_iter = 30
    cat_boost_iter = 35
    
## apply the other optimizations to both the lassoed X-train adn the x-train
    ## 
    lasso_opti = lasso_optimization(x234_array, y234_array, [0.01, 0.08,0.12, 0.15,1,10])
    # output of lasso optimization is a dataframe of with all the attempts, pull out the best 
    ## min score function???
    lasso_bestvalue = find_best_value(lasso_opti)
    # get the lassoed array and the list of its features
    ## lasso features for later
    [lassoed_features, lassoed_X_Train] = get_lassoed_array(lasso_bestvalue, features)
    
    # Ridge Regression
    ridge_all_features = ridge_optimization(x234_array, y234_array, ridge_iter)
    ridge_lasso_features = ridge_optimization(lassoed_X_Train, y234_array, ridge_iter)
    
    ridge_all_features['opti_run']= 'ridge_all_features'
    ridge_lasso_features['opti_run']= 'ridge_lasso_features'
    
    # RandomForestRegressor_optimization
    ran_forest_all_features = RandomForestRegressor_optimization(x234_array, y234_array, ran_forest_iter)
    ran_forest_lasso_features = RandomForestRegressor_optimization(lassoed_X_Train, y234_array, ran_forest_iter)
    
    ran_forest_all_features['opti_run']= 'ran_forest_all_features'
    ran_forest_lasso_features['opti_run']= 'ran_forest_lasso_features'
    
    # SVRegressor_optimization
    #SVR_all_features = SVRegressor_optimization(x234_array, y234_array, SVR_iter)
    #SVR_lasso_features = SVRegressor_optimization(lassoed_X_Train, y234_array, SVR_iter)
    #
    #SVR_all_features['opti_run']= 'SVR_all_features'
    #SVR_lasso_features['opti_run']= 'SVR_lasso_features'
    
    # DecisionTreeRegressor_optimization
    Dec_Tree_all_features = DecisionTreeRegressor_optimization(x234_array, y234_array, dec_tree_iter)
    Dec_Tree_lasso_features = DecisionTreeRegressor_optimization(lassoed_X_Train, y234_array, dec_tree_iter)
    
    Dec_Tree_all_features['opti_run']= 'Dec_Tree_all_features'
    Dec_Tree_lasso_features['opti_run']= 'Dec_Tree_lasso_features'
    
    # CatBooster_optimization
    catBoost_all_features = catboosterregressor_optimization(X_train_scaled, y_train, number_of_trials=cat_boost_iter, iter_numnber=150)
    catBoost_lasso_features = catboosterregressor_optimization(lassoed_X_Train, y_train, number_of_trials=cat_boost_iter, iter_numnber=150)
    
    catBoost_all_features['opti_run']= 'catBoost_all_features'
    catBoost_lasso_features['opti_run']= 'catBoost_lasso_features'
    
    concat_all_data = pd.concat([
        ridge_all_features, 
        ridge_lasso_features,
        ran_forest_all_features,
        ran_forest_lasso_features,
        #SVR_all_features,
        #SVR_lasso_features,
        Dec_Tree_all_features,
        Dec_Tree_lasso_features,
        catBoost_all_features,
        catBoost_lasso_features
        ])
    
    concat_best_data = pd.concat([
        find_best_value(ridge_all_features), 
        find_best_value(ridge_lasso_features),
        find_best_value(ran_forest_all_features),
        find_best_value(ran_forest_lasso_features),
        #find_best_value(SVR_all_features),
        #find_best_value(SVR_lasso_features),
        find_best_value(Dec_Tree_all_features),
        find_best_value(Dec_Tree_lasso_features),
        find_best_value(catBoost_all_features),
        find_best_value(catBoost_lasso_features)
        ])
    
    
# Elastic net
# XGBoost
# CatBoost
    
    # pull the concat best data
    # pull out the features that were lassoed 
    return concat_best_data, lassoed_features


In [5]:
## USEFUL FUNCTIONS
def find_best_value(output_from_opti):
    bestValue = output_from_opti[output_from_opti['combined_score']==min(output_from_opti['combined_score'])]
    return bestValue

# take list of coefficients - if more than zero 
# keep the name of the poly feature
# keep the non-zero poly features values
def get_lassoed_array(a_lasso_opti, column_names):
    ## pull the list of coefficients -- some funnny stuff needs to be done because of array stuff, basically reset_index
    coefficient_list = list(a_lasso_opti['coefficients'].reset_index()['coefficients'][0]) 

    ### creature a dictionary of features and coefficients and then turn into dataframe
    feature_coefficient_dict = {'feature':[], 'coefficient':[]}
    for (feaI, coeI) in zip(featurenames, coefficient_list): 
        if coeI != 0: 
            feature_coefficient_dict['feature'].append(feaI)
            feature_coefficient_dict['coefficient'].append(coeI)
            
    ## coefficient ----- important for later to maybe pull out 
    coefficient_df = pd.DataFrame(feature_coefficient_dict).sort_values('coefficient', ascending=False)
    feature_selection = feature_coefficient_dict['feature']
    lassoed_dataframe = pd.DataFrame(X_train, columns= column_names)[feature_selection]
    #### with the coefficients create a new np.array to 
    lassoed_x_train = np.array(pd.DataFrame(X_train, columns= column_names)[feature_selection])
    ## output is the coefficient list for the lasso and the new lassoed_x_train
    return [feature_coefficient_dict['feature'], lassoed_x_train]

In [6]:
## feature engineering -- create function to pull out coefficients/features
## add a gradient descent
def lasso_optimization(x123_train, y123_train, alpha_list):
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[], 'alpha':[], 'coeff_used':[], 'coefficients':[], 'intercept':[]}

    # initialize and train model with (default value) alpha = 0.5
    # for i in [0.1, 1, 10,100,1000, 10000, 33000,66000, 100000, 330000,660000,1000000, 3300000,6600000,10000000,33000000,66000000,100000000]:
    for i in alpha_list:

        model = Lasso(alpha=i, max_iter=int(2000))
        model.fit(x123_train, y123_train)
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        ## use cross validation for 
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        coeff_used = np.sum(model.coef_!=0)
        coefficients = model.coef_
        intercept = model.intercept_
        dicKeys = ['mean_MAPE','mean_r2','mean_MSE','std_MAPE','std_r2','std_MSE','score','alpha','coeff_used', 'coefficients', 'intercept'] 
        dicValues = [mean_MAPE, mean_r2, mean_MSE, std_MAPE, std_r2, std_MSE, len(score), i, coeff_used, coefficients, intercept]
        for (keyi, vali) in zip(dicKeys, dicValues):
            eval_metrics_dictionary[keyi].append(vali)
        #eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        #eval_metrics_dictionary['mean_r2'].append(mean_r2)
        #eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        #eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        #eval_metrics_dictionary['std_r2'].append(std_r2)
        #eval_metrics_dictionary['std_MSE'].append(std_MSE)
        #eval_metrics_dictionary['score'].append(len(score))
        #eval_metrics_dictionary['alpha'].append(i)
        #eval_metrics_dictionary['coeff_used'].append(coeff_used)

    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    #df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    return df_framework
#return df_framework.sort_values('combined_score')

In [7]:
def ridge_optimization(x123_train, y123_train, number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[],'alpha': [], 
                               'intercept' : [], 'tol': [], 
                               'solver':[], 'no_of_coefficients':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        alpha = trial.suggest_float("alpha", 0, 100)
        intercept = trial.suggest_categorical("fit_intercept", [True, False])
        tol = trial.suggest_float("tol", 0.001, 1, log=True)
        solver = trial.suggest_categorical("solver", ["auto", "svd","cholesky", "lsqr", "saga", "sag"])

        ## MODEL SELECTION
        ## Create Model
        model = Ridge(alpha=alpha,fit_intercept=intercept,tol=tol,solver=solver)
        ## Fit Model
        model.fit(x123_train, y123_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['alpha'].append(alpha)
        eval_metrics_dictionary['intercept'].append(intercept)
        eval_metrics_dictionary['tol'].append(tol)
        eval_metrics_dictionary['solver'].append(solver)
        eval_metrics_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MSE*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="RidgeRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    return df_framework

In [8]:
def RandomForestRegressor_optimization(x123_train, y123_train,number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[],
        'n_estimators': [], 
        'max_depth': [],
        'min_samples_split': [],
        'min_samples_leaf':[]
        }
    #randoforreg_dictionary = {'n_estimators': [], 'max_depth': [],'min_samples_split': [],'min_samples_leaf':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        n_estimators = trial.suggest_int("n_estimators", 10, 200, log=True)
        max_depth = trial.suggest_int("max_depth", 2, 32)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

        # Create and fit random forest model
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42,
            )
        model.fit(x123_train, y123_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        #coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['n_estimators'].append(n_estimators)
        eval_metrics_dictionary['max_depth'].append(max_depth)
        eval_metrics_dictionary['min_samples_split'].append(min_samples_split)
        eval_metrics_dictionary['min_samples_leaf'].append(min_samples_leaf)
        #randoforreg_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MSE*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="RandomForestRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    trial = study2.best_trial
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    return df_framework


In [9]:

def SVRegressor_optimization(x123_train, y123_train,number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[],'kernel' :[],'C' :[],'degree' :[],'coef0' :[],'gamma' :[]}
    def objective(trial):
        ## HYPER PARAMETER TUNING
        degree = trial.suggest_int("degree", 3, 8)
        C = trial.suggest_int("C", 1, 10)
        coef0 = trial.suggest_float("coef0", 0.01, 10)
        gamma = trial.suggest_categorical("gamma", ["auto", 'scale'])
        kernel = trial.suggest_categorical("kernel", ['poly'])
        # ['linear', 'poly', 'rbf', 'sigmoid'])


        # Create and fit random forest model
        model = SVR(
            kernel=kernel,
            C=C,
            degree=degree,
            coef0=coef0,
            gamma=gamma,
            )
        model.fit(x123_train, y123_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        #coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['degree'].append(degree)
        eval_metrics_dictionary['C'].append(C)
        eval_metrics_dictionary['coef0'].append(coef0)
        eval_metrics_dictionary['gamma'].append(gamma)
        eval_metrics_dictionary['kernel'].append(kernel)
        #randoforreg_dictionary['no_of_coefficients'].append(coeff_used)
        #return mean_MSE*mean_MAPE*(1-mean_r2)*(1-mean_r2)
        return mean_MSE*(1-mean_r2)

    study2 = optuna.create_study(study_name="SVRRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    return df_framework

In [10]:
def DecisionTreeRegressor_optimization(x123_train, y123_train,number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[], 'max_depth': [],'min_samples_split': [],'min_samples_leaf': [],'random_state':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        max_depth = trial.suggest_int("max_depth", 2, 32)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
        random_state = trial.suggest_int("random_state", 0, 50)

        # Create and fit random forest model
        model = DecisionTreeRegressor(
                max_depth=max_depth,
                min_samples_leaf=min_samples_leaf, 
                min_samples_split=min_samples_split,
                random_state=random_state
                )
        model.fit(x123_train, y123_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, x123_train, y123_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_root_mean_squared_error'), cv=3)      
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_root_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_root_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        #coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['max_depth'].append(max_depth)
        eval_metrics_dictionary['min_samples_split'].append(min_samples_split)
        eval_metrics_dictionary['min_samples_leaf'].append(min_samples_leaf)
        eval_metrics_dictionary['random_state'].append(random_state)
        #randoforreg_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MSE*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="DecisionTreeRegressor Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    return df_framework

In [11]:
def catboosterregressor_optimization(x123_train, y123_train, number_of_trials, iter_numnber):
    
    
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[],
                               'learning_rate' : [], 
                               'depth' : [], 
                               'l2_leaf_reg':[]
                               #'l2_leaf_reg': [], 
                               #'min_child_samples':[]
                               }

    def objective(trial):
        param = {
            'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.1),
            'depth': trial.suggest_int("depth", 2, 10),
            'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 0.2, 3)
        }
        model = CatBoostRegressor(**param, iterations=iter_numnber)        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set        
        ## problem arises here because cross validate will fit the model each time.....
        ## MUST BUILD YOUR OWN CROSS VALIDATE SCORE, SO THAT THE MODEL IS 
        score = eric_cross_validate(model, X_train_scaled, y_train, 5)
        
        mean_MSE = np.mean(score['RMSE'])
        mean_r2 = np.mean(score['r2_score'])
        mean_MAPE = np.mean(score['MAPE'])
        std_MSE = np.std(score['RMSE'])
        std_r2 = np.std(score['r2_score'])
        std_MAPE = np.std(score['MAPE'])
                
        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        eval_metrics_dictionary['learning_rate'].append(param['learning_rate'])
        eval_metrics_dictionary['depth'].append(param['depth'])
        eval_metrics_dictionary['l2_leaf_reg'].append(param['l2_leaf_reg'])

        return mean_MSE*(1-mean_r2)
        #return mean_MSE

    study2 = optuna.create_study(study_name="catboosterregressor Optimization")
    study2.optimize(objective, n_trials=number_of_trials,n_jobs=-1, timeout=24000)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    df_framework['combined_score'] = abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    return df_framework

In [12]:
#catboosterregressor_optimization(X_train_scaled, y_train, number_of_trials=50, iter_numnber=150)

In [13]:
## Eric Cross Validate
def eric_cross_validate(infunc_model, x345_train, y345_train, cv):
## we want to partition the data....
    x_split = np.array_split(x345_train, cv)
    y_split = np.array_split(y345_train, cv)
    
    cross_val_dict = {'RMSE': [], 'r2_score': [], 'MAPE':[]}
    for i in list(range(len(list(x_split)))): 
        
        ## create our CV splits
        infuncXList = list(x_split)
        infuncYList = list(y_split)
        temp_X_val_split = infuncXList.pop(i)
        temp_X_train_split = np.concatenate(x_split)  
        temp_Y_val_split = infuncYList.pop(i)
        temp_Y_train_split = np.concatenate(y_split) 
        
        ## fit the model 
        inmodel = infunc_model
        inmodel.fit(temp_X_train_split, temp_Y_train_split, verbose = 0, early_stopping_rounds = 200) 
        in_ypred = inmodel.predict(temp_X_val_split)
        
        ## pull evaluation metrics 
        rmse = mean_squared_error(temp_Y_val_split, in_ypred,squared=False)
        r2 = r2_score(temp_Y_val_split, in_ypred)
        MAPE = mean_absolute_percentage_error(temp_Y_val_split, in_ypred)
        
        ## add values to the dictionary
        cross_val_dict['RMSE'].append(rmse)
        cross_val_dict['r2_score'].append(r2)
        cross_val_dict['MAPE'].append(MAPE)

    return cross_val_dict

## create a model...off the train data
## test it against the val data
eric_cross_validate(model, X_train_scaled, y_train, 5)

NameError: name 'model' is not defined